In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import requests
import pickle
import sys
import re
import csv

In [2]:
crawled_liste = []

In [3]:
class Article:
    def __init__(self, title, lead, datum, author, text, url):
        self.title = title
        self.lead = lead
        self.datum = datum
        self.author = author
        self.text = text
        self.url = url

In [31]:
def inhalter():
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    try:
        title = soup.find("h1").get_text().strip()
    except AttributeError:
        title = ""
    
    try:
        lead = soup.find(class_="field-subhead").get_text().strip()
    except AttributeError:
        lead = ""
    
    try:
        author = soup.find(class_="authorline__link--archiv").get_text().strip()
    except AttributeError:
        author = ""
    
    try:
        date = soup.find(class_="field-issue-ref").get_text().strip()
        date = date[-11:].strip()
    except AttributeError:
        date = ""
    
    fintext = str()
    text = soup.find_all("p", class_=None)
    for i in text[:-5]:
        if author not in i.get_text():
            i = i.get_text()
            i = i.replace("\n", " ")
            i = i.strip()
            fintext += i
            
    ding_url = page
    
    crawled = Article(title, lead, date, author, fintext, ding_url)
    crawled_liste.append(crawled)

In [23]:
driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.maximize_window()

In [24]:
driver.get("https://www.woz.ch/")

In [25]:
#manual login
#p = pickle.dump(driver.get_cookies() , open("woz_cookies.pkl","wb"))

In [26]:
cookies = pickle.load(open("woz_cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
    
driver.refresh()

In [27]:
with open("woz_output.bin", "rb") as data:
    ueb_pages = pickle.load(data)

In [32]:
for page in ueb_pages:
    time.sleep(2)
    try:
        driver.get(page)
        inhalter()
    except TimeoutException:
        print("TimeOut")
        pass    

In [33]:
with open('woz_art1.csv', 'w', newline='', encoding="utf8") as csvfile:
    blogwriter = csv.writer(csvfile, delimiter='|', quotechar='"')


    for article in crawled_liste:
        blogwriter.writerow( [article.title, article.lead, article.datum, article.author, article.text, article.url] )

In [ ]:
driver.close()